## Create NCSBI Raw Crime Datasets by County

** This program downloads North Carolina crime rate datasets from http://crimereporting.ncsbi.gov/Reports.aspx and saves them as .csv files. **

   1. This program will dowload specific crime data by year from the correct URL.
   2. It will loop the though all 100 North Carolina counties to get each specific year's data.
   3. We generate a new csv file and save specific year data for the entire state by county.
    
Contributors:
* Bin Yu
* Nuoya Rezsonya
* Jake Drew

Created:
* 04/2019

In [1]:
# How to install missing packages into the current jupyter environment:

#import sys
#!conda install --yes --prefix {sys.prefix} requests
#!conda install --yes --prefix {sys.prefix} beautifulsoup4

In [2]:
# import libaraies
import requests
import urllib.request
import time
import csv
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [3]:
year = 2017
outputPath = 'C:/Users/Jake/Documents/GitHub/EducationDataNC/' + str(year)\
           + '/Raw Datasets/CrimeDataByCounty' + str(year) + '.csv'
# root URL for the crime rate for the requested year including 10 years dataset for each county, 
#but we only want a single year per file
#rootPath = 'http://crimereporting.ncsbi.gov/public/' + str(year) + '/CrimeTrds/IndexRateCoTrd/report1/'
rootPath = 'http://crimereporting.ncsbi.gov/public/2017/CrimeTrds/IndexRateCoTrd/report1/'

In [4]:
# Get the header
#url = 'http://crimereporting.ncsbi.gov/public/' + str(year) + '/CrimeTrds/IndexRateCoTrd/report1/1.htm'
url = 'http://crimereporting.ncsbi.gov/public/2017/CrimeTrds/IndexRateCoTrd/report1/1.htm'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
# fourth table has the header
table = soup.findAll('table')[4]
headers = []
header = []
header.append('County')

# header is in the first row
for val in table.find_all('tr')[0].find_all('td'):
    #print(val.text)
    header.append(val.text)
    
headers.append(header)

In [5]:
def getCrimeDataByYear(url,yearToExtract):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    title = soup.findAll('table')[3]
    county = title.findAll('td')[0].text + ' County'
    
    table = soup.findAll('table')[4]
   
    for row in table.find_all('tr')[1:]:

        data =[]
        data.append(county)
        firstCol = True
        for val in row.find_all('td'):
            if(firstCol==True):
                yearVal = int(val.text)
            fieldVal = val.text
            if(fieldVal == '\xa0'):
                fieldVal = ''
            data.append(fieldVal)
            firstCol=False
        #print(yearVal,year)

        if(yearVal==yearToExtract):
            #print('true')
            rows.append(data)

In [7]:
# Loop for all of the counties to get the data for specific year
rows = []
for i in range(1,101):
    url = rootPath+str(i)+'.htm'
    getCrimeDataByYear(url,year)

In [8]:
#Create csv file from all of the crime data for the year requested.
#  credit: https://www.programiz.com/python-programming/working-csv-files

csv.register_dialect('myDialect',
                     delimiter = ',',
                     quotechar = '"',
                     quoting=csv.QUOTE_ALL,
                     skipinitialspace=True)
# write the data to csv file
with open(outputPath, 'w',newline='\n', encoding='utf8') as file:
    writer = csv.writer(file, dialect="myDialect")
    writer.writerows(header for header in headers if header)
    writer.writerows(row for row in rows if row)
file.close()